In [3]:
import matplotlib.pyplot as plt
from albumentations import Flip, ShiftScaleRotate, RandomRotate90, RandomCrop
from src.datasets import HEDataset,ConsepDataset,ConsepDataset_Fast
import os 
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "5"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.set_num_threads(3)

In [4]:
import albumentations as A
import random
import numpy as np


In [5]:
data_transform = A.Compose([A.Flip(p=0.3),
                            A.Affine(p=0.3),
                            A.Rotate(p=0.3),
                            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=15, val_shift_limit=10, p=0.3),
                            A.GaussianBlur(3, p=0.3),
                            A.GaussNoise(30, p=0.3)],
                            
                            keypoint_params=A.KeypointParams(format='yx'),
                            additional_targets={'mask0': 'mask',
                                                'mask1': 'mask',
                                                'mask2': 'mask'})

/home/wzy/.conda/envs/general/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


In [6]:
MoNuSegdisplay_set = ConsepDataset(data_dir = './CoNSeP',transform=data_transform,option = 'Train')
# MoNu_0 = MoNuSegdisplay_set[1]
# fig,axes = plt.subplots(1,3, figsize=(30,10))

# axes[0].imshow(MoNu_0["images"].numpy().transpose(1,2,0)) 
# axes[1].imshow(MoNu_0["region"].numpy())
# axes[2].imshow(MoNu_0["obj"].numpy(),cmap='jet')


#### Training Process

In [7]:
import glob,os
from trainval import trainval
import matplotlib.pyplot as plt
from haven import haven_utils as hu

In [8]:
# Define the dataset you want to train on, you can type 'TNBC' or 'MoNuSegTrainingData'
datadir = './CoNSeP/' 

# the exp_dict is the configuration of the model which has already been saved. 
# It consists of the initial learning rate, max epoch, model name and optimizer
# The saved one looks like this
# {
#     "batch_size": 1,



#     "lr": 0.001,
#     "max_epoch": 20,
#     "model": {
#         "name": "penet"
#     },
#     "optimizer": "adam"
# }
exp_dict = hu.load_json('Experiment_Exp/exp_config_mynet_1.json')
exp_dict["batch_size"] = 64
exp_dict["max_epoch"] = 1

datadir = './CoNSeP/' 
random_seed = random.randint(0, 20201009)
train_set = ConsepDataset_Fast(data_dir=datadir,
                            n_classes=exp_dict["n_classes"],

                            
                            transform=data_transform,
                            option="Train",
                            random_seed=random_seed,
                            obj_option=None,
                            patch_size=256,
                            bkg_option=None)
# folddir_10 = glob.glob(os.path.join(datadir, '*_fold'))

# for folddir in folddir_10:
#     savedir_base = os.path.join(folddir,'Result')
#     os.makedirs(savedir_base,exist_ok=True)
savedir_base = "./Result"
os.makedirs(savedir_base,exist_ok=True)
trainval(exp_dict, savedir_base, datadir, reset=False, num_workers=8)

Experiment saved in ./Result/9cd2fe45a2b63c0b3d2d2865b76acfec


/home/wzy/.conda/envs/general/lib/python3.7/site-packages/albumentations/augmentations/blur/transforms.py:185: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  "blur_limit and sigma_limit minimum value can not be both equal to 0. "


Starting experiment at epoch 0


  0%|          | 0/3375 [00:00<?, ?it/s]

logits:
tensor([-6.2903, -6.2204, -6.2118,  ...,  4.1285,  4.1678,  4.2084],
       device='cuda:0', grad_fn=<Unique2Backward0>)
cam_output:
tensor([-2.7376, -2.7264, -2.6896,  ...,  2.7354,  2.7768,  2.7889],
       device='cuda:0', grad_fn=<Unique2Backward0>)


...

Training Loss: 0.4272:   1%|          | 34/3375 [00:38<57:58,  1.04s/it]

logits:
tensor([-5.1356, -5.1356, -5.1271,  ...,  5.1885,  5.1908,  5.1932],
       device='cuda:0', grad_fn=<Unique2Backward0>)
cam_output:
tensor([-3.8573, -3.8376, -3.8286,  ...,  1.9505,  1.9508,  1.9511],
       device='cuda:0', grad_fn=<Unique2Backward0>)


Training Loss: 0.4245:   1%|          | 35/3375 [00:39<58:27,  1.05s/it]

logits:
tensor([-6.4264, -6.4099, -6.3973,  ...,  5.9103,  5.9118,  5.9134],
       device='cuda:0', grad_fn=<Unique2Backward0>)
cam_output:
tensor([-3.7341, -3.7327, -3.7314,  ...,  2.0608,  2.0640,  2.0646],
       device='cuda:0', grad_fn=<Unique2Backward0>)


KeyboardInterrupt: 

#### Testing and Visualize

In [ ]:
from src import models
from src.datasets import HEDataset,ConsepDataset
# from trainval import loadsplittedfolds
import albumentations as A
import os,glob
import torch

datadir = './CoNSeP/' 
# folddir_10 = glob.glob(os.path.join(datadir, '*_fold'))

# folddir = folddir_10[0]

exp_dict = hu.load_json('Experiment_Exp/exp_config_mynet_1.json')

test_transform = A.Compose([A.Resize(256,256)], 
                          keypoint_params = A.KeypointParams(format='yx'), 
                          additional_targets = {'mask0':'mask', 
                                                'mask1':'mask'})

# tr_idx, val_idx, ts_idx = loadsplittedfolds(folddir)
test_set = test_set = ConsepDataset(data_dir = datadir, 
                                transform = test_transform,
                                option = "Test")

model = models.get_model(exp_dict['model'],exp_dict = exp_dict,train_set = test_set)
model.load_state_dict(torch.load('Result/v33/model_best.pth'))

RuntimeError: Error(s) in loading state_dict for Newnet:
	Missing key(s) in state_dict: "detection_head.0.weight", "detection_head.0.bias". 
	Unexpected key(s) in state_dict: "detection_head.fpn.P5_1.weight", "detection_head.fpn.P5_1.bias", "detection_head.fpn.P5_2.weight", "detection_head.fpn.P5_2.bias", "detection_head.fpn.P4_1.weight", "detection_head.fpn.P4_1.bias", "detection_head.fpn.P4_2.weight", "detection_head.fpn.P4_2.bias", "detection_head.fpn.P3_1.weight", "detection_head.fpn.P3_1.bias", "detection_head.fpn.P3_2.weight", "detection_head.fpn.P3_2.bias", "detection_head.fpn.P6.weight", "detection_head.fpn.P6.bias", "detection_head.fpn.P7_2.weight", "detection_head.fpn.P7_2.bias", "detection_head.regressionModel.conv1.weight", "detection_head.regressionModel.conv1.bias", "detection_head.regressionModel.conv2.weight", "detection_head.regressionModel.conv2.bias", "detection_head.regressionModel.conv3.weight", "detection_head.regressionModel.conv3.bias", "detection_head.regressio

In [ ]:
index = 2
test_im = test_set[index]['images'][None]
model.eval()
test_logit,regression,classification = model.model_base(test_im.to(model.device))


#### 分割结果的可视化

In [ ]:
test_prob = test_logit.sigmoid()
test_seg = torch.argmax(test_prob, dim = 1)
fig, axes = plt.subplots(1,3, figsize = (30,10))
axes[0].imshow(test_im[0].detach().cpu().numpy().transpose(1,2,0))
axes[1].imshow(test_set[index]['gt'],vmax = 7, vmin = 0)
axes[2].imshow(test_seg[0].detach().cpu().numpy(),vmax = 7, vmin = 0)

#### 第1,2个loss是正常的，现在怀疑第三个loss的第二部分有一定的问题，可能还是和预处理的过程有一定的关系
- v1: 图像监督 + 点监督 + obj loss 
- v2: 图像监督 + 点监督
- v3: 只有点监督，基本上只有按照颜色的预测
- v4：加上了obj部分的loss，感觉得到了一定的优化，但是也不是很明显
- v5: 加上了图片级别的监督信息（是否有细胞核存在）
- v6：用point当做region来计算region loss